# Übung 6: Transfer learning

Neuronal Netze benötigen oft eine große Menge an Trainingsdaten, damit es nicht zu overfitting kommt. Transfer learning erlaubt es, mit relativ geringen Datenmenge dennoch erfolgreiche große Netze zu trainieren. Dabei verwendet man ein bereits auf einen anderen Datensatz (z.b. ImageNet) vortrainiertes Netzwerk, und ersetzt nur das letzte Layer durch ein neues. In dieser Übung geht es darum, ein Netzwerk für die Erkennung von Geparden und Leoparden in der freien Wildbahn zu trainineren. 

## Daten laden

Lade die Daten hier herunter: http://tonic.imp.fu-berlin.de/cv_data/data.tar.gz

Die Daten wurde bereits in ein Trainings- und Validierungsset geteilt. Die Ordnerstruktur ist wie bei vielen Bildklassifierungsdatensetzen so aufgebaut. Es gibt zwei Unterordner für die Trainings- und Validierunsdaten. In diesen Ordnern liegen dann jeweils alle Bilder von einer Klasse in einem Unterordner mit dem Namen der Klasse.

Ein Beispiel: Die Trainingsbilder für die Klasse "cheetah" liegen in dem Unterordner train/cheetah

Diese Orderstruktur wird auch von dem in keras enhaltenen ImageDataGenerator unterstützt.

In [1]:
import os
import tensorflow as tf
import numpy as np
import keras
import pathlib

import warnings

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

from keras.backend.tensorflow_backend import set_session, get_session
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras import optimizers
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import VGG16, decode_predictions
#from keras.applications.inception_v3 import InceptionV3, decode_predictions
from keras.preprocessing import image
from keras.models import load_model
from keras.models import Model

from PIL import Image

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config = config))
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
batch_size = 32
image_input_size = (224, 224)
data_path = './data/'

prob_drop_conv = 0.5              
pool_size = (2, 2) 
nb_epoch = 150

opt = optimizers.RMSprop(lr = 0.000001, rho = 0.9)

In [3]:
train_data_path = os.path.join(data_path, 'train')
val_data_path = os.path.join(data_path, 'val')

classes = ('unknown', 'cheetah', 'leopard')

train_gen = ImageDataGenerator(horizontal_flip=True).flow_from_directory(
    train_data_path, 
    target_size = image_input_size,
    classes = classes,
    batch_size = batch_size)

val_gen = ImageDataGenerator(horizontal_flip=False).flow_from_directory(
    val_data_path, 
    target_size = image_input_size,
    classes = classes,
    batch_size = batch_size,
    shuffle = False)

Found 17857 images belonging to 3 classes.
Found 1915 images belonging to 3 classes.


## Training ohne transfer learning

Trainiere zuerst ein kleines Classifer-Netzwerk ohne transfer learning. Falls du keine Grafikkarte hast, solltest du nicht die volle Auflösung (siehe Variable image_input_size) verwenden, da das Training sonst zu lange dauert. Eine Bildgröße von 32x32 Pixeln wäre zum Beispiel möglich.

In [ ]:
#CONV MODEL
model = Sequential()
steps_per_epoch=len(pool_size)/batch_size

# FIRST CONV LAYER
model.add(Convolution2D(8, 3, 3, border_mode = 'same', activation = 'relu', input_shape = [224,224,3]))
model.add(MaxPooling2D(pool_size = pool_size, strides=(2,2), border_mode = 'same'))
model.add(Dropout(prob_drop_conv))

# SECOND CONV LAYER
model.add(Convolution2D(16, 3, 3, border_mode = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size, strides = (2,2), border_mode = 'same'))
model.add(Dropout(prob_drop_conv))

# THIRD CONV LAYER
model.add(Convolution2D(4, 3, 3, border_mode = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size, strides = (2,2), border_mode = 'same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# FIRST FC LAYER
model.add(Dense(625, activation = 'relu'))
model.add(Dropout(prob_drop_conv))

# SECOND FC LAYER
model.add(Dense(3, activation = 'softmax'))

model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()
print(model.summary())

history = model.fit_generator(train_gen,steps_per_epoch, nb_epoch = nb_epoch, shuffle = False, verbose = 1)

In [ ]:
# Evaluate
evaluation = model.evaluate_generator(val_gen, 1)

print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

Erstelle eine Confusion matrix basierend auf den Ausgaben des Netzes für die Validierungsdaten und berechne den ROC AUC für die Klasse cheetah. Du kannst hierfür optional die scikit-learn Bibliothek verwenden.

In [ ]:
num_of_test_samples = 1915

Y_pred = model.predict_generator(val_gen, num_of_test_samples // batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
conf_matrix = confusion_matrix(val_gen.classes, y_pred)
print(conf_matrix)

print('')

print('Classification Report')
print(classification_report(val_gen.classes, y_pred, target_names=['unknown', 'cheetah', 'leopard']))

print('')

print('ROC AUC Score')
#print(sklearn.metrics.roc_auc_score(y_true, y_score))
0.75

## Pretrained network

Lade nun ein auf Imagenet vortrainiertes Netzwerk und klassifiziere damit die Validierungsdaten. Eine Anleitung für keras findest du hier: https://keras.io/applications

Du kannst selber entscheiden, welche Netzwerkarchitektur du verwendest.

In [33]:
vgg16_model = VGG16(weights='imagenet')
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Da der ImageNet-Datensatz auch die Klassen cheetah und leopard enthält, können wir sogar ohne transfer learning das vortrainierte Netzwerk evaluieren. Interpretiere alle Klassen außer cheetah und leopard als unknown und berechne wie im vorherigen Schritt die Confusion matrix und den ROC AUC score für die Klasse cheetah.

In [58]:
vgg_predictions = vgg16_model.predict_generator(val_gen, steps=len(val_gen))

In [59]:
valid_labels = val_gen.classes
print(valid_labels.shape, vgg_predictions.shape)

(1915,) (1915, 1000)


In [60]:
print(classes)

('unknown', 'cheetah', 'leopard')


In [65]:
vgg16_predictions = vgg_predictions.argmax(axis = 1)

# replace all unknown classes with a zero
np.place(vgg16_predictions, vgg16_predictions == 1, 0)
np.place(vgg16_predictions, vgg16_predictions == 2, 0)
np.place(vgg16_predictions, vgg16_predictions == 293, 1)
np.place(vgg16_predictions, vgg16_predictions == 288, 2)
np.place(vgg16_predictions, vgg16_predictions > 2, 0)

# Matrix
vgg_conf_matrix = confusion_matrix(valid_labels, vgg16_predictions)


# ROC AUC Score
valid_labels_auc_value = np.copy(valid_labels)
roc_curve_predictions = np.copy(vgg_predictions)

np.place(valid_labels_auc_value, valid_labels_auc_value == 2, 0)

roc_score = roc_auc_score(valid_labels_auc_value, roc_curve_predictions[:, 0])

In [66]:
print('Confusion Matrix')
print(vgg_conf_matrix)

print('')

print('ROC AUC Score')
print(roc_score)



Confusion Matrix
[[ 352    9    3]
 [1205  228    7]
 [  84   25    2]]

ROC AUC Score
0.4237616959064327


## Transfer learning

Das vortrainierte Netzwerk kann nun mit unseren Daten weitertrainiert werden. Ersetze dafür das letzte Layer in dem Netzwerk mit einem Dense Layer mit 3 Ausgaben für unsere Klassen cheetah, leopard und unknown. Du kannst selbst entscheiden, ob du nun das komplette Netzwerk mit trainierst oder nur das neu eingefügte, letzte Layer.

Auch hierfür kannst du dich wieder an der keras Anleitung orientieren: https://keras.io/applications

In [15]:
custom_model = Sequential()
for layer in vgg16_model.layers:
    custom_layer = layer
    custom_layer.trainable = False
    custom_model.add(custom_layer)
    
custom_model.layers.pop()
custom_model.add(Dense(3, activation = "softmax"))
custom_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [17]:
custom_model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001),
              metrics = ['accuracy'])

W1210 21:58:14.361592 4661394880 deprecation_wrapper.py:119] From /Users/ced/miniconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [18]:
custom_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [22]:
custom_model.fit_generator(
        train_gen,
        steps_per_epoch = 1800 // batch_size,
        epochs=50,
        validation_data = val_gen,
        validation_steps = 250 // batch_size)

W1210 21:59:12.251341 4661394880 deprecation.py:323] From /Users/ced/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
56/56 [==============================] - 269s 5s/step - loss: 1.0715 - acc: 0.5725 - val_loss: 1.0049 - val_acc: 0.9375
Epoch 2/50
56/56 [==============================] - 263s 5s/step - loss: 1.0123 - acc: 0.6540 - val_loss: 0.9036 - val_acc: 1.0000
Epoch 3/50
56/56 [==============================] - 274s 5s/step - loss: 0.9654 - acc: 0.6490 - val_loss: 0.8157 - val_acc: 1.0000
Epoch 4/50
56/56 [==============================] - 254s 5s/step - loss: 0.9308 - acc: 0.6378 - val_loss: 0.7489 - val_acc: 1.0000
Epoch 5/50
56/56 [==============================] - 251s 4s/step - loss: 0.9009 - acc: 0.6406 - val_loss: 0.6834 - val_acc: 1.0000
Epoch 6/50
56/56 [==============================] - 252s 4s/step - loss: 0.8704 - acc: 0.6562 - val_loss: 0.6414 - val_acc: 1.0000
Epoch 7/50
56/56 [==============================] - 250s 4s/step - loss: 0.8443 - acc: 0.6557 - val_loss: 1.1461 - val_acc: 0.4932
Epoch 8/50
56/56 [==============================] - 250s 4s/step - loss: 0.8243 - a

In [31]:
custom_model.save('models/custom-model.h5')
print('Model saved!')
custom_model.save_weights('models/custom-model-weights.h5')
print('Model-Weights saved!')

Model saved!
Model-Weights saved!


In [ ]:
#custom_model.load_weights('models/custom-model-weights.h5')
#print(Model-Weights loaded!)

Evaluiere das so trainierte Netzwerk wie in den letzten beiden Aufgaben.

In [24]:
print(classes)

('unknown', 'cheetah', 'leopard')


In [54]:
custom_model_predictions = custom_model.predict_generator(val_gen, steps=len(val_gen))

In [55]:
valid_labels_2 = val_gen.classes

# ROC AUC Score
# set all cheetahs to 1, all other to zero:
valid_labels_auc_vaule_2 = np.copy(valid_labels_2)
roc_curve_predictions_2 = np.copy(custom_model_predictions)
np.place(valid_labels_auc_vaule_2, valid_labels_auc_vaule_2 == 2, 0)

roc_score_2 = roc_auc_score(valid_labels_auc_vaule_2, roc_curve_predictions_2[:, 1])

custom_model_predictions = custom_model_predictions.argmax(axis = 1)

In [56]:
custom_model_conf_matrix = confusion_matrix(valid_labels_2, custom_model_predictions)

print('Confusion Matrix')
print(custom_model_conf_matrix)

print('')

print('ROC AUC Score')
print(roc_score_2)

Confusion Matrix
[[   0  364    0]
 [ 233 1207    0]
 [   0  111    0]]

ROC AUC Score
0.34604970760233916


# Auswertung

Beschreibe kurz qualitativ die Resultate. Wie unterscheiden sich die trainierten Netzwerke, zum Beispiel im Bezug auf die Genauigkeit oder die Laufzeit? Welche Entscheidungen musstest du bei der Erfüllung der Aufgaben treffen und warum hast du dich für den von dir gewählten Weg entschieden?